## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json
import os

In [2]:
# Variables
season = "2021-22"


## Paths for Files



In [ ]:
source_path = os.getcwd()

cleaned_data_path =  os.path.join(source_path, "data" , season , "Cleaned_data")
merged_data_path =  os.path.join(source_path, "data" , season , "Merged_data")
team_data_path =  os.path.join(source_path, "data" , season , "Team_data")

## Import files

In [3]:
all_events = pd.read_csv(os.path.join(cleaned_data_path, 'all_events.csv'), parse_dates= ["deadline_time"])
all_players = pd.read_csv(os.path.join(merged_data_path, 'all_players_merged.csv'), parse_dates= ["news_added"]) 

## Functions

In [4]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

## My Team

In [5]:
#### Get my fantasy team lineup
now = datetime.now()
my_team_id = '272984'
current_gw = all_events[all_events['deadline_time'] < now]['gameweek_id'].max()

my_team_url = 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/'.format(my_team_id,str(current_gw))

my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)


In [6]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost',
       'photo', 'PPG', 'second_name', 'selected_by_percent', 'status',
       'team_id', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'thre

In [7]:
#all_players = all_players[['chance_of_playing_next_round', 'chance_of_playing_this_round', 'position_id', 'ep_next','ep_this', 'player_id', 'player_news', 'news_added', 'player_price', 'selected_by_percent', 'player_status','team_id', 'web_name']]

all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,...,NaN,NaN,1.333333,0.288889,-0.155556,4,ARS,1210,1270,GKP
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
2,0.0,0.0,463748,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
3,NaN,NaN,225321,0,0,6,-6,3,1,0.0,...,NaN,NaN,5.026596,0.980392,0.588235,4,ARS,1210,1270,GKP
4,NaN,NaN,220682,0,0,-1,1,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.512821,4,ARS,1210,1270,GKP


In [8]:
my_team

,element,position,multiplier,is_captain,is_vice_captain
0,270,1,1,False,False
1,256,2,1,False,False
2,370,3,1,False,False
3,431,4,1,False,False
4,110,5,1,False,False
5,701,6,1,False,False
6,196,7,1,False,False
7,420,8,1,False,False
8,518,9,1,False,False
9,700,10,2,True,False


In [9]:
my_team = my_team.merge(all_players, how = 'left', left_on="element", right_on="player_id")
my_team.drop(["element","position"], axis = 1, inplace = True)

In [10]:
my_team.sort_values(by=['player_type_id' , 'selected_by_percent'])['web_name']



0               de Gea
11             Sánchez
4               Lowton
3                Saïss
2             Reguilón
1              Cancelo
14    Alexander-Arnold
5           Kulusevski
6             Raphinha
12                Saka
7                Bowen
13               Salah
9             Weghorst
8                Broja
10                Kane
Name: web_name, dtype: object

In [11]:
my_team.to_csv (os.path.join(team_data_path, 'my_team.csv'), index = False, header=True)

In [12]:
my_team

,multiplier,is_captain,is_vice_captain,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,1,False,False,NaN,NaN,51940,0,0,3,-3,...,NaN,NaN,4.000000,0.754717,0.377358,4,MUN,1210,1270,GKP
1,1,False,False,100.0,100.0,121145,-1,1,11,-11,...,NaN,NaN,5.952492,0.830986,0.549296,5,MCI,1350,1370,DEF
2,1,False,False,100.0,100.0,199249,0,0,3,-3,...,NaN,NaN,4.864865,0.773585,0.396226,4,TOT,1210,1270,DEF
3,1,False,False,100.0,100.0,107613,0,0,0,0,...,NaN,NaN,4.302789,0.840000,0.440000,4,WOL,1200,1220,DEF
4,1,False,False,NaN,NaN,68983,0,0,-1,1,...,NaN,NaN,3.326733,0.704545,0.250000,2,BUR,1060,1050,DEF
5,1,False,False,NaN,NaN,445044,0,0,0,0,...,NaN,NaN,5.625000,0.666667,0.333333,4,TOT,1210,1270,MID
6,1,False,False,100.0,100.0,219961,0,0,0,0,...,2.0,NaN,4.862288,0.676923,0.369231,3,LEE,1090,1100,MID
7,1,False,False,NaN,NaN,178186,0,0,6,-6,...,NaN,NaN,6.045987,0.802817,0.521127,4,WHU,1200,1200,MID
8,1,False,False,100.0,100.0,440323,0,0,4,-4,...,NaN,NaN,5.459016,0.648148,0.277778,3,SOU,1090,1100,FWD
9,2,True,False,100.0,75.0,120202,0,0,0,0,...,NaN,NaN,4.825175,0.707692,0.400000,2,BUR,1060,1050,FWD
